In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
data_df = pd.read_csv('Data\\recs2009_public.csv')
data_df.dropna(axis=0,inplace=True)
data_df.drop(columns=["DOEID"],inplace=True)
data_df.reset_index(inplace=True, drop=True)

cols = data_df.columns.tolist()

In [3]:
for i in range(len(data_df.index)):
    for col in cols:
        if col != "METROMICRO" and col != "UR":
            if data_df.at[i, col] < 0:
                data_df.at[i, col] = 0

        elif col == "UR":
            if data_df.at[i, col] == "U":
                data_df.at[i, col] = 0
            else:
                data_df.at[i, col] = 1
        else:
            if data_df.at[i, col] == "METRO":
                data_df.at[i, col] = 0
            if data_df.at[i, col] == "MICRO":
                data_df.at[i, col] = 1
            if data_df.at[i, col] == "NONE":
                data_df.at[i, col] = 2
            

In [4]:
data_df

,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,Climate_Region_Pub,...,SCALEEL,KAVALNG,PERIODNG,SCALENG,PERIODLP,SCALELP,PERIODFO,SCALEFO,PERIODKR,SCALEKER
0,2,4,12,2,2471.679705,4742,1080,4953,1271,4,...,0,0,0,0,0,0,0,0,0,0
1,4,10,26,2,8599.172010,2662,199,2688,143,5,...,0,1,1,0,0,0,0,0,0,0
2,1,1,1,5,8969.915921,6233,505,5741,829,1,...,0,3,5,3,0,0,0,0,0,0
3,2,3,7,2,18003.639600,6034,672,5781,868,1,...,3,3,5,3,0,0,0,0,0,0
4,1,1,1,3,5999.605242,5388,702,5313,797,1,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12076,2,3,9,2,10219.802880,7671,332,7784,451,1,...,0,3,5,3,0,0,0,0,0,0
12077,3,7,21,4,8689.074445,2085,2844,2520,2286,2,...,0,1,1,0,0,0,0,0,0,0
12078,2,4,10,2,4697.213624,7935,472,8074,600,1,...,0,1,1,0,0,0,0,0,0,0
12079,4,8,23,2,13155.678790,5834,770,5989,957,1,...,0,1,1,0,0,0,0,0,0,0


In [5]:
# BTUEL is KWH just in another unit of measurement and KWH*** are a separation of KWH on specific tasks such as airconditioning.

KWH_cheat = [i for i in data_df.columns if(("KWH" in i and len(i)>=4 or "BTUEL" in i))]
print(KWH_cheat)

data_df.drop(columns = KWH_cheat, inplace = True)

['KWHSPH', 'KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH', 'BTUEL', 'BTUELSPH', 'BTUELCOL', 'BTUELWTH', 'BTUELRFG', 'BTUELOTH']


In [6]:
data_df.to_csv("./Data/processed.csv", index=False)

In [7]:
to_normalize = data_df[data_df.columns.difference(['METROMICRO', 'UR', 'KWH'])]

kwh = data_df['KWH']
data_df.drop(columns = 'KWH', inplace=True)

df_to_normalize = StandardScaler().fit_transform(to_normalize)
df_to_normalize = pd.DataFrame(df_to_normalize)
df_to_normalize.columns = data_df.columns.difference(['METROMICRO', 'UR'])


data_df = pd.concat([df_to_normalize, kwh],axis = 1)
print(data_df)

# data_df.to_csv("./Data/normalized_processed.csv", index=False)

       ACOTHERS   ACROOMS  ADQINSUL    AGEAUD  AGECDRYER  AGECENAC  AGECWASH  \
0     -0.128751  1.579809 -1.131006 -0.117893  -0.548990 -0.393733 -0.474097   
1     -0.128751  0.089509  1.513663 -0.117893  -0.421785  2.007708 -0.474097   
2     -0.128751 -1.102730  0.191329 -0.117893  -0.612593 -0.583320 -0.603639   
3     -0.128751  0.983689 -1.131006 -0.117893  -0.548990 -0.456929 -0.538868   
4     -0.128751 -1.102730  0.191329 -0.117893  -0.485388 -0.583320 -0.474097   
...         ...       ...       ...       ...        ...       ...       ...   
12076 -0.128751  0.387569  0.191329 -0.117893  -0.421785 -0.393733 -0.409326   
12077 -0.128751 -0.804670  1.513663 -0.117893  -0.612593 -0.393733 -0.603639   
12078 -0.128751  1.281749 -1.131006 -0.117893   1.995114  2.070904  2.051972   
12079 -0.128751 -0.208551  0.191329 -0.117893   2.058716 -0.393733  2.116743   
12080 -0.128751 -1.102730  1.513663 -0.117893  -0.485388 -0.583320 -0.474097   

          AGEDW   AGEFRZR  AGEFRZR2  ..